In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
import hook_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *
import csv

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [65]:
model = HookedTransformer.from_pretrained("gpt2-large")

with open('data/gpt2_large_spectrum_v2.pkl', 'rb') as f:
    sample_df = pickle.load(f)

with open('data/gpt2_large_spectrum_bins_v2.pkl', 'rb') as f:
    bin_edges = pickle.load(f)

def read_labels(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return {row[0]: row[1] for row in csv.reader(f)}
    return {}

labels = read_labels('data/gp2_large_spectrum_labels_v2.csv')
sorted_index_labels = sorted([(int(index), int(label)) for index, label in labels.items()])
sorted_labels = np.array([label for index, label in sorted_index_labels])

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


In [66]:
sample_df['label'] = sorted_labels

In [ ]:
text = load_dataset("stas/openwebtext-10k", split="train")

In [84]:
high_an_samples = sample_df[sample_df["L31N892 Activation"] == pd.Interval(6.473, 6.823, 'right')]

for i in range(10):
    sample = high_an_samples.iloc[i]
    print(model.to_str_tokens(model.to_tokens(text['text'][sample['prompt_index']])[0, :sample['token_index']+2]))

['<|endoftext|>', '"', 'An', 'or', 'ak', '"', ' redirect', 's', ' here', '.', ' For', ' other', ' uses', ',', ' see', ' An']
['<|endoftext|>', 'J', 'oh', 'ann', ' Reich', 'hart', ' (', ' –', ' )', ' was', ' a', ' state', '-', 'appointed', ' judicial', ' execution', 'er', ' in', ' Bav', 'aria', ',', ' Germany', ' from', ' 1924', '–', '19', '46', '.', ' During', ' the', ' Nazi', ' Germany', ' era', ',', ' he', ' executed', ' numerous', ' people', ' who', ' were', ' sentenced', ' to', ' death', ' for', ' resisting', ' National', ' Socialism', '.[', '2', ']', '\n', '\n', 'Life', ' and', ' career', ' [', ' edit', ' ]', '\n', '\n', 'J', 'oh', 'ann', ' Reich', 'hart', ' was', ' born', ' in', ' W', 'ichen', 'bach', ' near', ' W', 'ör', 'th', ' an', ' der', ' Don', 'au', ' into', ' a', ' family', ' of', ' Bav', 'arian', ' kn', 'ackers', ' and', ' execution', 'ers', ',', ' including', ' his', ' uncle', ' Franz', ' X', 'aver', ' Reich', 'hart', ' and', ' brother', ' Michael', ',', ' that', ' went

In [80]:
sample_df[sample_df["L31N892 Activation"] == pd.Interval(6.473, 6.823, 'right')]


,prompt_index,token_index,token,next_token,next_is_an,activation,L31N892 Activation,label
950,3796,14,766,1052,True,6.741928,"(6.473, 6.823]",3
951,4666,142,400,281,True,6.822765,"(6.473, 6.823]",3
952,6580,668,4608,393,False,6.539912,"(6.473, 6.823]",2
953,194,359,2063,281,True,6.578099,"(6.473, 6.823]",3
954,5761,437,1394,281,True,6.672470,"(6.473, 6.823]",3
955,2307,50,3829,13,False,6.694689,"(6.473, 6.823]",2
956,900,146,284,2319,False,6.638927,"(6.473, 6.823]",2
957,3076,940,1262,281,True,6.487067,"(6.473, 6.823]",3
958,532,196,546,1115,False,6.603206,"(6.473, 6.823]",2
959,4990,588,329,262,False,6.732648,"(6.473, 6.823]",2


In [85]:
grouped_total = sample_df.groupby('L31N892 Activation', as_index=False).count()['label']

# len(grouped_total)
# print(sample_df[sample_df['label'] == 3].groupby('L31N892 Activation', as_index=False).count()['label'].fillna(0))
grouped_3 = sample_df[sample_df['label'] == 3].groupby('L31N892 Activation', as_index=False).count()['label'] / grouped_total
# print(sample_df[sample_df['label'] == 3].groupby('L31N892 Activation', as_index=False).count()['label'])
# print(sample_df[sample_df['label'] == 3][sample_df['L31N892 Activation'] == 17].groupby('L31N892 Activation', as_index=False).count()['label'])
# print(sample_df[sample_df['label'] == 2][sample_df['L31N892 Activation'] == 17].groupby('L31N892 Activation', as_index=False).count()['label'])
# print(sample_df[sample_df['label'] == 1][sample_df['L31N892 Activation'] == 17].groupby('L31N892 Activation', as_index=False).count()['label'])
# print(len(grouped_3), grouped_total, )
grouped_3 = grouped_3.fillna(0)
# print(grouped_3)
grouped_2 = sample_df[sample_df['label'] == 2].groupby('L31N892 Activation', as_index=False).count()['label'] / grouped_total
# print(len(grouped_2))
grouped_2 = grouped_2.fillna(0)
# print(len(grouped_2))
grouped_1 = sample_df[sample_df['label'] == 1].groupby('L31N892 Activation', as_index=False).count()['label'] / grouped_total
# print(len(grouped_1))
grouped_1 = grouped_1.fillna(0)
# print(len(grouped_1))

df = pd.DataFrame({
    'L31N892 Activation': grouped_3.index,
    'Next token is \' an\'' : grouped_3.values,
    'Next token is plausible \' an\'': grouped_2.values,
    'Next token is not \' an\'': grouped_1.values
})

# grouped.head()
# grouped[grouped['label'] == 3].head()

# Cut off tails, 20 data points per


/tmp/ipykernel_1797/2188503789.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_1797/2188503789.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_1797/2188503789.py:13: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_1797/2188503789.py:17: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=Fal

In [86]:
# sample_df[sample_df['L31N892 Activation'] == 17].groupby('L31N892 Activation', as_index=False).count()

In [87]:
# print(df[df['L31N892 Activation'] == 17])


In [88]:
grouped_melted = df.melt(id_vars=['L31N892 Activation'], value_vars=['Next token is \' an\'', 'Next token is plausible \' an\'', 'Next token is not \' an\''], 
                            var_name='status', value_name='Proportion of Labels')
# print(grouped_melted[grouped_melted['L31N892 Activation'] == 17])
# print(grouped_total)


In [89]:
# grouped = sample_df.groupby('L31N892 Activation', as_index=False)

# grouped['Next token is \' an\''] = grouped_3

# grouped['Next token is plausible \' an\''] = grouped_2
# grouped['Next token is not \' an\''] = grouped_1

fig = px.area(grouped_melted, x='L31N892 Activation', y='Proportion of Labels', color='status', line_shape='spline')
tickvals = np.arange(len(bin_edges) - 1)
ticktext = [f"{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}" for i in tickvals]

fig.update_layout(
    yaxis=dict(range=[0, 1]),
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ),
)
fig.show()

In [72]:
grouped.head()

,prompt_index,token_index,token,next_token,next_is_an,activation,L31N892 Activation,label
0,2183,279,10744,44378,False,-0.047074,0,1
1,76,606,287,2805,False,-0.093404,0,2
2,994,245,198,198,False,-0.166820,0,2
3,2182,250,36144,416,False,-0.074979,0,2
4,1346,49,12,28875,False,-0.002914,0,1
...,...,...,...,...,...,...,...,...
1066,669,679,416,281,True,12.921400,17,3
1067,391,689,4608,281,True,12.731512,17,3
1068,1919,688,2641,281,True,12.707108,17,3
1070,201,661,423,281,True,13.994400,18,3
